# Archery Capstone
This capstone project was created in support of the CodeYou program to showcase the materials covered in the Data Analysis pathway. 
The code blocks below will walk you through the following:<br>
1. Accessing three layers created within ArcGIS Online to collect archery related stats from 3D target events<br>
2. Pulling these layers down to data frames to work with separately from the authoritative data within ArcGIS Online<br>
3. Cleaning and joining the data so it can be used for futher analysis and graphic creation<br>
4. Creating graphics and shot summaries for the archer to reference and analyze for trends

## Data Collection
Data was created and collected using esri's ArcGIS Online platform in support of this project. Three distinct layers will be referenced for this project:<br>
1.  A lookup table was created containing the various 3D target animals along with information on the color, size, name, and target pattern.<br> 
2.  A Survey123 was created to collect shoot conditions for each tournament.  Data was collected to capture the shoot name, shoot location (indoor or outdoor), shoes (minimal, hiking boots, sandals, gym shoes), temperature, weather conditions present (sunny, cloudy, humid, windy, foggy, light rain, heavy rain, stormy), and personal conditions (well rested, tired, sick, sore, hungry, full).<br>
3.  A Survey123 was created to collect several details for each shot on the individual target and appearance (animal, range, position, elevation, lighting), shot execution (confidence, release details, execution details), and placement (score, aim placement, actual arrow impact). 

### Import Modules

This section imports the various modules required for the code to execute. 

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import pandasql as psql
import os
import requests
from IPython.display import display
import plotly.express as px
import ipywidgets as widgets
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
print("Modules Imported")

### Download AGOL layers to dataframes

This section connects to ArcGIS Online and downloads two of the three datasets to dataframes for cleaning and analysis. The final dataset is then converted to a dataframe from a CSV. 

In [ ]:
# Connect to ArcGIS Online
# gis = GIS("https://sso.maps.arcgis.com/")
gis = GIS()

# Define ItemIDs for AGOL layers
Shot = "f0314c505e83428d8ce6894a8d73eb09"
Shoot = "5fa46f5f47e841b1b2454271cd346d8f"

# Search for the layers by title
layer3_Shoot = gis.content.get(Shoot)
layer2_Shot = gis.content.get(Shot)
#print (layer2_Shot.type)
#print (layer3_Shoot.type)

# Access the first layer in each item
feature_layer2_Shot = layer2_Shot.layers[0]
feature_layer3_Shoot = layer3_Shoot.layers[0]
# print (feature_layer2_Shot)
# print (feature_layer3_Shoot)

# Query the layers to get all features
features2 = feature_layer2_Shot.query()
features3 = feature_layer3_Shoot.query()

# Convert features to pandas DataFrames
df2_Shot = features2.sdf
df3_Shoot = features3.sdf

# Read the AnimalLookup CSV to a dataframe
cwd = os.getcwd()
print(cwd)
data_dir = os.path.join(cwd, "Data")
print(data_dir)
animalLookup_path = f"{data_dir}\\AnimalLookup.csv"
df1_Animal = pd.read_csv(animalLookup_path)

print("Dataframes imported")

In [ ]:
# Now that our data has been downloaded to dataframes, let's make sure we are able to see the resulting columns and rows. 
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)


# Explore the columns and data of each data frame
print(f"Animal Lookup Dataframe: {df1_Animal.columns}")
display(df1_Animal)
print(f"Shot Report Out Dataframe: {df2_Shot.columns}")
display(df2_Shot)
print(f"Shoot Details: {df3_Shoot.columns}")
display(df3_Shoot)

### Combine and clean dataframes using Pandas

1. Populate fields<br>
    - Score to Scored Ring<br>

In [ ]:
# A few changes were made to the schema mid data collection - let's clean up some of the columns.  

# For the Shot Report Out dataframe, lets cleanup the score column
# Set column 'scored_ring' equal to column 'score' where column 'scored_ring' is NaN.  Due to column differences, we need to set the column type as well.
df2_Shot['scored_ring'] = df2_Shot['scored_ring'].astype(float)
df2_Shot['score'] = df2_Shot['score'].astype(float)
df2_Shot['scored_ring'] = df2_Shot['scored_ring'].fillna(df2_Shot['score'])
#display(df2_Shot)
print("Field Updated")

2. Lets combine the three separate dataframes into a single dataframe using the merge function from pandas. This will be done based on common fields and data values between the dataframes.

In [ ]:
#  Here are our three dataframes, for reference:  df1_Animal, df2_Shot, df3_Shoot

#  First, we will do a merge on df1_Animal to df2_Shot (df1_Animal.TargetName = df2_Shot.target).  Then, we will do another merge between the new dataframe and the df3_Shoot data frame (new.date_and_time = df3_Shoot.date_and_time_of_Shoot) based on the date portion of the datetime fields. 

# Merge df1_Animal and df2_Shot on 'TargetName' and 'target'
merged_df = pd.merge(df2_Shot, df1_Animal,  left_on='target', right_on='Target Name', how='left')
#display(merged_df)


# Extract the date portion from 'date_and_time' and 'date_and_time_of_Shoot'
merged_df['date'] = pd.to_datetime(merged_df['date_and_time']).dt.date
df3_Shoot['date'] = pd.to_datetime(df3_Shoot['date_and_time_of_shoot']).dt.date

# Merge the resulting DataFrame with df3_Shoot on the date portion of the datetime strings
final_merged_df = pd.merge(merged_df, df3_Shoot, on='date', how='inner')

# # Drop the extra 'date' column
final_merged_df.drop(columns=['date'], inplace=True)
#display(final_merged_df)

print("Dataframes merged")

3. Clean dataframes using pandas
<p>The survey123 mobile applications used to collect information on the overall shoot details and shot report out used functionality called "Multi-Select" fields.  This meant the archer was able to select multiple values for a single question if the values were all relevant, streamlining the data entry process. For example, when recording the position of the animal, the archer could select if it was Quartered_to, Quartered_From, Broadside, Leaning_Forward, or Leaning_Back - all from the same question.  </p>

<p>When multiple answers are selected, the survey concatenates the answers into a string in a single field.  In order to perform analysis on the recorded values for the multi-select fields, we need to break out and add unique columns to our data frames.  To do this, we'll create new columns in our merged dataframe and use string operations to populate the columns based on the values present in the concatenated fields.</p>

Create flag fields on the multi-select questions below:
-  Adjustment (adjustment)
    -  Increase_Yardage, Decrease_Yardage, Sight_Left, Sight_Right
-  Position (position)
    -  Quartered_to, Quartered_From, Broadside, Leaning_Forward, Leaning_Back
-  Shot Placement (shot_placement)
    -  On_Aim_Point, High, Low, Left, Right, Miss, Kick_Out
-  Shot Execution (shot_execution)
    -  Unsure, Felt_Good, Bad_Shoulder, Bad_Release, Movement_in_Shot, Unable_to_Center_Bubble, Dip, Push_Up, Glare, other
-  Release Details (release_details)
    -  Too_Soon, Fast, Slow, Off_Face, Hand_Moved
-  Weather Conditions Present (weather_conditions_present)
    -  Sunny, Cloudy, Humid, Windy, Foggy, Light_Rain, Heavy_Rain, Stormy
-  Personal Conditions (personal_conditions)
    -  Well_Rested, Tired, Sick, Sore, Hungry, Full

In [ ]:
# Define the values to check
position = ['Quartered_to','Quartered_From','Broadside','Leaning_Forward','Leaning_Back']
shot_placement = ['On_Aim_Point','High','Low','Left','Right','Miss, Kick_Out']
shot_execution = ['Unsure','Felt_Good','Bad_Shoulder','Bad_Release','Movement_in_Shot','Unable_to_Center_Bubble','Dip','Push_Up','Glare','other']
release_details = ['Too_Soon','Fast','Slow','Off_Face','Hand_Moved']
weather_conditions_present = ['Sunny','Cloudy','Humid','Windy','Foggy','Light_Rain','Heavy_Rain','Stormy']
personal_conditions = ['Well_Rested','Tired','Sick','Sore','Hungry','Full']

# Function to create new columns based on values in a specific column
def create_columns_based_on_values(df, column_name, values_list):
    if column_name in df.columns:
        for value in values_list:
            #df[value] = df[column_name].apply(lambda x: 1 if value in x else 0)
            df[value] = df[column_name].fillna('').apply(lambda x: 1 if value in x else 0)
    else:
        print(f"Column '{column_name}' does not exist in the DataFrame.")

# Create new columns for each category
create_columns_based_on_values(final_merged_df, 'shot_execution', shot_execution)
create_columns_based_on_values(final_merged_df, 'position', position)
create_columns_based_on_values(final_merged_df, 'release_details', release_details)
create_columns_based_on_values(final_merged_df, 'shot_placement', shot_placement)
create_columns_based_on_values(final_merged_df, 'weather_conditions_present', weather_conditions_present)
create_columns_based_on_values(final_merged_df, 'personal_conditions', personal_conditions)


# Drop columns not useful within analysis and graph creation:
#print(f"Final Dataframe pre-drop: {final_merged_df.columns}")
final_merged_df.drop(columns=['globalid_x', 'CreationDate_x', 'Creator_x','target_order','untitled_question_3_other', 'score', 'adjustment_made', 'adjustment','SHAPE_x', 'OBJECTID','objectid_y', 'globalid_y', 'CreationDate_y','Creator_y', 'EditDate_y', 'Editor_y','SHAPE_y'], inplace=True)
#print(f"Final Dataframe post-drop: {final_merged_df.columns}")

display(final_merged_df)

### Create graphics from shooting statistics
The code block below will create a dashboard where you can compare any two variables within the dataframe to see graphs and patterns.  Below are recommendations to try - as well as what they mean. 
- Target and range, Violin graph type
    - Shows the spread of ranges the different animals are typically placed at.  
- Target Size and range, Scatter graph type
    - Shows the spread of ranges a target is typically placed at - from this graph you can see the medium sized targets are normally placed outside of 33 yards. 
- aim_placement and range, scatter
    - Shows the archers spread of aiming at the center of the ten ring or the lower 12 as the range increases.
- Shot-placement on aim_placement, histogram, count
    - This shows how often the archer hit what they were aiming at - in this graph trends can be seen that indicate the archer was either on target or high and left.
- Slow on Target Size, Bar, Sum
    -  This graph indicates the archer may be holding longer on big targets which causes a slow release and more movement in the shot. 
- range and date_and_time_of_shoot, box, count
    - This graph shows the average spread of ranges at the four events records were collected at. 



In [ ]:
df = final_merged_df

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Interactive Data Visualization", style={'color': 'teal'}),
    html.Label("Choose X-axis variable", style={'color': 'green', 'font-weight': 'bold'}),
    dcc.Dropdown(
        id='x-axis',
        options=[{'label': col, 'value': col} for col in df.columns],
        value=df.columns[0]
    ),
    html.Label("Choose Y-axis variable", style={'color': 'green', 'font-weight': 'bold'}),
    dcc.Dropdown(
        id='y-axis',
        options=[{'label': col, 'value': col} for col in df.columns],
        value=df.columns[1]
    ),
    html.Label("Choose graph type", style={'color': 'beige', 'font-weight': 'bold'}),
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Scatter', 'value': 'scatter'},
            {'label': 'Line', 'value': 'line'},
            {'label': 'Bar', 'value': 'bar'},
            {'label': 'Histogram', 'value': 'histogram'},
            {'label': 'Box', 'value': 'box'},
            {'label': 'Violin', 'value': 'violin'},
            {'label': 'Pie', 'value': 'pie'},
            {'label': 'Density Heatmap', 'value': 'density_heatmap'},
            {'label': 'Density Contour', 'value': 'density_contour'}
        ],
        value='scatter'
    ),
    html.Label("Aggregation Function (only for applicable graphs)", style={'color': 'beige'}),
    dcc.Dropdown(
        id='agg-func',
        options=[
            {'label': 'Sum', 'value': 'sum'},
            {'label': 'Count', 'value': 'count'}
        ],
        value='sum'
    ),
    dcc.Graph(id='graph')
])

# Callback to update graph based on user input
@app.callback(
    Output('graph', 'figure'),
    [Input('x-axis', 'value'), Input('y-axis', 'value'), Input('graph-type', 'value'), Input('agg-func', 'value')]
)
def update_graph(x_axis, y_axis, graph_type, agg_func):
    if graph_type == 'scatter':
        fig = px.scatter(df, x=x_axis, y=y_axis)
    elif graph_type == 'line':
        fig = px.line(df, x=x_axis, y=y_axis)
    elif graph_type == 'bar':
        if agg_func == 'sum':
            fig = px.bar(df, x=x_axis, y=y_axis)
        elif agg_func == 'count':
            fig = px.bar(df, x=x_axis, y=df.groupby(x_axis)[y_axis].transform('count'))
    elif graph_type == 'histogram':
        if agg_func == 'sum':
            fig = px.histogram(df, x=x_axis, y=y_axis)
        elif agg_func == 'count':
            fig = px.histogram(df, x=x_axis)
    elif graph_type == 'box':
        fig = px.box(df, x=x_axis, y=y_axis)
    elif graph_type == 'violin':
        fig = px.violin(df, x=x_axis, y=y_axis)
    elif graph_type == 'pie':
        fig = px.pie(df, names=x_axis, values=y_axis)
    elif graph_type == 'funnel':
        fig = px.funnel(df, x=x_axis, y=y_axis)
    elif graph_type == 'density_heatmap':
        fig = px.density_heatmap(df, x=x_axis, y=y_axis)
    elif graph_type == 'density_contour':
        fig = px.density_contour(df, x=x_axis, y=y_axis)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)



### Future Work
1.  Identify further parameters for performing analysis
2.  Fine tune the shoot report mobile survey to collect additional informatiton (i.e. which direction is the target facing), populate defaults, improve the data entry process, and allow for open ranges / judging. 
3.  Create a PDF or Word Report for daily shoot summaries
    
    